In [3]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    mape
)
import plotly.graph_objects as go

In [4]:
# seq_length = 10
# window_size = 5
# batch_size = 2
# hidden_size = 4
# output_size = 3

# sequence = list(fibonacci_generator(seq_length))
# X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
# X_train, y_train = X[0:int(len(X) * 0.8)], y[0:int(len(X) * 0.8)][-1]
# X_test, y_test = X[int(len(X) * 0.8):], y[int(len(X) * 0.8):][-1]

# verbosity = 1000

# max_mape = 1e-3
# learn_until_meet_mape = True

# model = GRUModel(window_size, hidden_size, output_size)

# # Параметры обучения
# lr = 1e-4
# epochs = 5000

# model.train(X_train, y_train, lr, epochs,
#             True, max_mape, verbosity)
# Проверим результат после обучения
# y_pred, _, _ = model.forward(X_train)
# print("Предсказание после обучения:\n", y_pred)
# print("Истинное значение:\n", y_train)
# print(f"MAPE: {mape(y_train, y_pred) * 100:.4f}%")

# print('TEST')
# y_pred, _, _ = model.forward(X_test)
# print("Предсказания на тестовой выборке:\n", y_pred)
# print("Истинное значение на тестовой выборке:\n", y_test)
# print(f"MAPE: {mape(y_test, y_pred) * 100:.4f}%")  

In [6]:
# Relation between images amount and training epochs
experiments_amount = 5
epochs = list(range(1000, 36000, 1000))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 50
window_size = 5
batch_size = 2
hidden_size = 4
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_epochs.append(max_epochs)
        mapes_sum += network.epochs        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

ValueError: non-broadcastable output operand with shape (5,4) doesn't match the broadcast shape (22,5,4)

In [ ]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()